# Part 1:

In [3]:
opstack = []

def opPop(): 
    return opstack.pop()

def opPush(value):
    opstack.append(value)
    return True

In [4]:
dictstack = [{}]

def dictPop():
    return dictstack.pop()

# add new dictionary
def dictPush():
    dictstack.append({})
    return True

def define(name, value):
    # add definition on top dicitonary
    dictstack[-1][name] = value
    return True

def lookup(name):
    # reverse to look at top of stack when iterating
    dictstack.reverse()
    for d in dictstack:
        r = d.get(name, False)
        if r is not False:
            # revert back to it's normal self
            dictstack.reverse()
            return r
        
    return False

In [5]:
def checkVar(value):
    value = str(value) 
    # find out if it's a variable or actual number
    if (value[0] == '-' and value[1:].isdigit()) or value.isdigit():
        value = int(value)
    elif value == 'false':
        return False
    elif value == 'true':
        return True
    else:
        value = lookup(value)
    return value

def add():    
    opPush(checkVar(opPop()) + checkVar(opPop()))
def sub():
    opPush(checkVar(opPop()) - checkVar(opPop()))
def mul():
    opPush(checkVar(opPop()) * checkVar(opPop()))
def div():
    opPush(checkVar(opPop()) / checkVar(opPop()))
def eq():
    opPush(checkVar(opPop()) == checkVar(opPop()))
def lt():
    opPush(checkVar(opPop()) > checkVar(opPop()))
def gt():
    opPush(checkVar(opPop()) < checkVar(opPop()))

In [6]:
def psand():
    opPush(opPop() and opPop())
def psnot():
    opPush(not opPop())
def psor():
    opPush(opPop() or opPop())

In [7]:
def dup():
    opPush(opstack[-1])
def exch():
    val1 = opPop()
    val2 = opPop()
    opPush(val1)
    opPush(val2)
def pop():
    opPop()

In [8]:
def dictz():
    opPush({})
def begin():
    top = opPop()
    # only move dictionaries
    if type(top) is dict:
        dictPush()
        return True
    else:
        print("ERROR: Can't move non-dictionary type to dictionary stack.")
        return False
    
def end():
    # pop dictionary stack
    dictPop()
    
def psDef():
    var1 = opPop()
    var2 = opPop()
    # is var2 a variable name
    if type(var2) is not str:
        return False
    # add variable 
    define(var2, var1)
    return True


In [9]:
def equals():
    print(opPop())
    
def stack():
    for item in opstack:
        print(item)

In [10]:
def testMath():
    opPush(1)
    opPush(2)
    add()
    if opPop() != 3: return False   
    opPush(5)
    opPush(5)
    mul()
    if opPop() != 25: return False
    opPush(2)
    opPush(4)
    sub()
    if opPop() != 2: return False
    opPush(2)
    opPush(4)
    div()
    if opPop() != 2: return False
    opPush(1)
    opPush(1)
    eq()
    if not opPop(): return False
    opPush(3)
    opPush(1)
    gt()
    if not opPop(): return False
    opPush(1)
    opPush(3)
    lt()
    if not opPop(): return False
    return True

def testLookup():
    opPush("n1")
    opPush(3)
    psDef()
    if lookup("n1") != 3: return False
    return True

def testBool():
    # test and
    opPush(True)
    opPush(False)
    psand()
    if opPop():
        return False

    #test not
    opPush(False)
    psnot()
    if not opPop():
        return False
    
    # test or
    opPush(False)
    opPush(False)
    psor()
    if opPop():
        return False
    
    return True

def testDupExch():
    opPush(1)
    dup()
    s = opPop()
    t = opPop()
    
    if not s == t == 1:
        return False
    
    opPush(5)
    opPush(10)
    exch()
    s = opPop()
    t = opPop()
    if t != 10 or s != 5:
        return False
    return True

# go on writing test code for ALL of your code here; think about edge cases, and 
# other points where you are likely to make a mistake.

# now an easy way to run all the test cases and make sure that they all return true
# is

testCases = [testLookup, testBool, testDupExch, testMath]
def testAll():
    for test in testCases:
        if not test(): return False
    return True

# but wouldn't it be nice to run all the tests, instead of stopping on the first failure,
# and see which ones failed
# How about something like:

testCases = [('lookup', testLookup), ('bools', testBool), ("dup&exch", testDupExch), ("math", testMath)]
def testAll():
    failedTests = [testName for (testName, testProc) in testCases if not testProc()]
    if failedTests:
        return ('Some tests failed', failedTests)
    else: return ('All tests OK')
testAll()

'All tests OK'

# Part 2:

In [11]:
def psif():
    carr = opPop()
    if opPop():
        interpret(carr)
def psifelse():
    second = opPop()
    first = opPop()
    if(opPop()):
        interpret(first)
    else:
        interpret(second)

In [12]:
keywords = {
    "def":psDef,
    "dictz":dictz,
    "begin":begin,
    "end":end,
    "stack":stack,
    "=":equals,
    "pop":pop,
    "exch":exch,
    "dup":dup,
    "lt":lt,
    "gt":gt,
    "eq":eq,
    "add":add,
    "sub":sub,
    "mul":mul,
    "div":div,
    "not":psnot,
    "and":psand,
    "or":psor,
    "if":psif,
    "ifelse":psifelse
}

In [32]:
import re

# checks for matching braces
def matchBraces(s):
    s= iter(s[0:])
    for c in s:
        if c == '{':
            if grouper(s):
                continue
            else:
                return False
        elif c == '}':
            print("Syntax error: missing a \'{0}\'".format('{'))
            return False
    return True
            
    
def grouper(it):
    for c in it:
        if c == '{':
            grouper(it)
        elif c == '}':
            return True
        prevWord = c
    print("Syntax error: missing a \'{0}\'".format('}'))
    return False

# tokenizes code and returns token array
def tokenize(s):
    l = re.findall("/?[a-zA-Z][a-zA-Z0-9_]*|[-]?[0-9]+|[}{]+|%.*|[^ \t\n]", s)
    # fix bug where a token ends up being '}{'
    for i in range(len(l)):
        if l[i] == '}{':
            l[i:i+1] = ['}', '{']
    
    if matchBraces(l):
        return l
    else:
        return False
    
# takes a list of tokens and returns a code array
def parse(tokens):
    return innerParse(iter(tokens[0:]))
            
def innerParse(it):
    l = []
    for c in it:
        if c == '}':
            return l
        elif c == '{':
            l.append(innerParse(it))
        else:
            l.append(c)
    return l

def interpret(code): # code is a code array
    for c in code:
        # code array
        if type(c) is list:
            opPush(c)
            continue
            
        func = keywords.get(c, c)
        
        # name constants, numbers, booleans
        if type(func) is str:
            if func[0] == '/':
                opPush(c[1:])
            else:
                func = checkVar(func)
                # code array
                if type(func) is list:
                    interpret(func)
                # booleans and constants
                else:
                    opPush(func)
        # callable function
        else:
            func()
        

def interpreter(s): # s is a string
    interpret(parse(tokenize(s)))

interpreter(
'''
/fact{ 
dictz begin
      /n exch def
         n 2 lt
         {1}
         {n -1 add fact n mul }
      ifelse
   end
}def { d}
6 fact =
'''
)
interpreter(
'''
true 
{-1}{1} 
ifelse
=
'''
)
interpreter(
'''
true {1} if
=
'''
)

720
-1
1
